# Unemployment by sex and age – monthly data

## Documentación

El conjunto de datos pertenece a Eurostat

Está disponible ne la siguiente url: https://ec.europa.eu/eurostat/databrowser/product/page/UNE_RT_M?lang=en

La documentación se puede encontrar en https://ec.europa.eu/eurostat/cache/metadata/en/une_rt_m_esms.htm#unit_measure1678720107776

Muestra la evolución del desempleo en los diferentes países de la unión europea.

Los campos del conjunto de datos son:
*   freq: frecuencia con la que se miden los datos. Todos los registros tienen el valor M (Mensualmente)
*   s_adj: ajuste estacional (SA, NSA, TC)
*   age: rango de edades en los que se ha realizado la medición TOTAL (todas las edades), Y25-74 (entre 25 y 74 años), Y_LT25 (Menos de 25 años)
*   unit: unidad en la que se ha realizado la medición PC_ACT (Percentage of Active Population), THS_PER (Thousand persons)
*   sex: sexo M (Masculino), F (Femenino), T (Todos)
*   geo: código de la unidad geográfica (país) en la que se ha realizado la medición
*   TIME_PERIOD: periodo en el que se realiza la medida con el formato YYYY-MM

## Limpieza

1) Carga del conjunto de datos

In [1]:
import pandas as pd

In [2]:
# Leer el archivo estat_une_rt_m.tsv como archivo de texto
with open('/content/estat_une_rt_m.tsv', 'r') as file:
    data = file.read()

# Reemplaza las tabulaciones y "/" por comas
data = data.replace('\t', ',').replace('/', ',')

# Guarda el contenido actualizado en unemployment.csv
with open('/content/unemployment.csv', 'w') as file:
    file.write(data)

In [3]:
df_unemployment = pd.read_csv('/content/unemployment.csv')

2) Se cambia la estructura del conjunto

In [4]:
# Realiza la conversión utilizando la función melt()
df = pd.melt(df_unemployment, id_vars=['freq', 's_adj', 'age', 'unit', 'sex', 'geo\TIME_PERIOD'],
                    var_name='date', value_name='tasa')

df = df.rename(columns={'geo\TIME_PERIOD': 'geo'})

3) Se limpian las columnas
 - Se la columna 'date' se convierte a tipo fecha
 - Se eliminan los caracteres no numéricos de la columna 'tasa' y se convierte a numérica
 - Se cambian los valores de la columna 'age'
 - Se cmabian los valores de la columna 'unit'

In [5]:
df['date'] = pd.to_datetime(df['date'])
df = df.loc[~df['tasa'].str.contains(': ')]
df['tasa'] = df['tasa'].str.replace(" p", "")
df['tasa'] = df['tasa'].str.replace(" d", "")
df['tasa'] = df['tasa'].str.replace(" b", "")
df['tasa'] = df['tasa'].str.replace(" u", "")
df['tasa'] = df['tasa'].astype(float)

df['age'] = df['age'].str.replace("TOTAL", "All")
df['age'] = df['age'].str.replace("Y25-74", "25-74")
df['age'] = df['age'].str.replace("Y_LT25", "0-25")

df['unit'] = df['unit'].str.replace("PC_ACT", "%PA")
df['unit'] = df['unit'].str.replace("THS_PER", "1.000P")

4) Se eliminan tanto los registros vacíos como los no necesarios

In [6]:
df = df.loc[df['unit'] != '1.000P']
df = df.loc[df['s_adj'] == 'SA']
df = df.drop(['freq', 's_adj','unit'], axis=1)
df = df.loc[df['geo'] != 'EA19']
df = df.loc[df['geo'] != 'EA20']
df = df.loc[df['geo'] != 'EU27_2020']
df = df.loc[df['geo'] != 'nan']
df = df.dropna(subset=['geo'])
print(df)

         age sex geo       date  tasa
673      All   F  DK 1983-01-01   8.5
681      All   F  FR 1983-01-01   9.0
684      All   F  IE 1983-01-01  14.4
686      All   F  IT 1983-01-01  11.2
687      All   F  JP 1983-01-01   2.7
...      ...  ..  ..        ...   ...
943503  0-25   M  SE 2023-04-01  19.8
943523  0-25   T  FI 2023-04-01  17.0
943527  0-25   T  IE 2023-04-01   7.9
943535  0-25   T  NL 2023-04-01   7.9
943540  0-25   T  SE 2023-04-01  18.8

[110217 rows x 5 columns]


5) Se define un diccionario de códigos de países y se aplica el cambio de códigos en la columna 'geo'


In [7]:
codigo_paises = {
    'AT': 'AUT',
    'BE': 'BEL',
    'BG': 'BGR',
    'CY': 'CYP',
    'CZ': 'CZE',
    'DE': 'DEU',
    'DK': 'DNK',
    'EE': 'EST',
    'EL': 'GRC',
    'ES': 'ESP',
    'FI': 'FIN',
    'FR': 'FRA',
    'HR': 'HRV',
    'HU': 'HUN',
    'IE': 'IRL',
    'IS': 'ISL',
    'IT': 'ITA',
    'JP': 'JPN',
    'LT': 'LTU',
    'LU': 'LUX',
    'LV': 'LVA',
    'MT': 'MLT',
    'NL': 'NLD',
    'NO': 'NOR',
    'PL': 'POL',
    'PT': 'PRT',
    'RO': 'ROU',
    'SE': 'SWE',
    'SI': 'SVN',
    'SK': 'SVK',
    'TR': 'TUR',
    'US': 'USA'
}

df['geo'] = df['geo'].map(codigo_paises)

6) Se crea un excel con los datos

In [8]:
df.to_excel('/content/unemployment.xlsx', index=False)